In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch

In [5]:
es = Elasticsearch(hosts="localhost:9200")

def parse(selector):
    return BeautifulSoup(driver.page_source, 'lxml', from_encoding='utf-8').select(selector)

def checkUrl(url):
    try:
        if es.get(index="community_data", id=url)['_id'] == url:
            print(url)
            print(es.get(index="community_data", id=url)['_id'])
            return True
    except:
        return False
# 해당 Page 게시물 URL 전체 수집
def getUrl(pageNum):
    sleep(1)
    e = []
    for i in range(1,pageNum+1):
        driver.get("https://lineagem.plaync.com/board/all/list?page="+str(i))
        for el in parse(publicUrl):
            if checkUrl(el['href']) == False:
                e.append(el['href'])
    print(len(e))
    return e

# 다음 Page로 이동
def nextPage():
    #sleep(1)
    num = int(parse(currentPage)[0].text)
    print(num+1)
    nextUrl = "https://lineagem.plaync.com/board/all/list?page="+str(num+1)
    print(nextUrl)
    driver.get(nextUrl)

def getTitle():
    sleep(1)
    #print(getUrl()[0])
    #driver.get(getUrl()[0])
    return parse(title)[0].text

def getBody():
    #sleep(1)
    textBody = "";
    for s in parse(body):
        #print(s)
        textBody += s.text.replace("\xa0", "")
    return textBody

def getDate():
    #sleep(1)
    textDate = ""
    textDate = parse(date)[0].text
    return textDate.replace(" ","").replace(".","-")[:10]

def getHits():
    #sleep(1)
    return parse(hits)[0].text.replace(",", "")

def getComments():
    #sleep(1)
    if len(parse(comments[0])) == 0:
        return 0
    else:
        return parse(comments)[0].text.replace(",", "")

def getRecommend():
    #sleep(1)
    return parse(recommend)[0].text.replace(",", "")

def check_index(opt=True):
        if not opt and es.indices.exists("community_data"):
            print("# 해당 인덱스 초기화를 위한 삭제")
            es.indices.delete("community_data")
        if not es.indices.exists("community_data"):
            print("# 해당 인덱스 미존재")
            settings = {
                "index": {
                "analysis": {
                    "tokenizer": {
                    "nori_tokenizer_mixed": {
                        "type": "nori_tokenizer",
                        "decompound_mode": "mixed"
                    }
                    },
                    "analyzer": {
                    "korean": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer_mixed",
                        "filter": [
                        "nori_readingform",
                        "lowercase",
                        "nori_part_of_speech_basic"
                        ]
                    }
                    },
                    "filter": {
                    "nori_part_of_speech_basic": {
                        "type": "nori_part_of_speech",
                        "stoptags": ["E", "IC", "J",
                                    "MAG", "MAJ", "MM",
                                    "SP", "SSC", "SSO", "SC", "SE",
                                    "XPN", "XSA", "XSN", "XSV",
                                    "UNA", "NA", "VSV"]
                    }
                    }
                }
                }
            }
            body = {
            "settings": settings,
            "mappings" : {"properties" :
                            {"title" : {"type" : "text", "analyzer" : "korean"},
                                "body" : {"type" : "text", "analyzer" : "korean"},
                                "date" : {"type": "date", "format": "yyyy-MM-dd"}
                            }}
            }
            es.indices.create(index="community_data", body=body)#인덱스 생성
            print("# 해당 인덱스 생성 완료")

def crawling(pageNum):
    es = Elasticsearch(hosts="localhost:9200")
    test = []
    test2 = []
    for i, url in enumerate(getUrl(pageNum)):
        driver.get(url)
        test.append({"create" : {"_index" : "community_data", "_id" : url}})
        test.append({
                    "community" : "리니지M 공식홈페이지",
                    "title" : getTitle(),
                    "date" : getDate(),
                    "body" : getBody(),
                    "hits" : int(getHits()),
                    "commments" : int(getComments()),
                    "recommand" : int(getRecommend())
                    })
        print(i+1, " " , url)
        if (i+1) % 100 == 0:
            es.bulk(index="community_data", body=test)
            es.indices.refresh(index="community_data")
            print(len(test))
            test.clear()
            print(len(test))
    es.bulk(index="community_data", body=test)
    es.indices.refresh(index="community_data")
    print(len(test))
    print(test)
    test.clear()
    print(len(test))
    

In [9]:
#check_index(opt=False)
crawling(12)

C:\Users\Playdata\Anaconda3\envs\encore\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


240
1   https://lineagem.plaync.com/board/server/view?articleId=1709228&page=1&viewMode=list&size=20
2   https://lineagem.plaync.com/board/server/view?articleId=1709227&page=1&viewMode=list&size=20
3   https://lineagem.plaync.com/board/server/view?articleId=1709226&page=1&viewMode=list&size=20
4   https://lineagem.plaync.com/board/free/view?articleId=1709225&page=1&viewMode=list&size=20
5   https://lineagem.plaync.com/board/server/view?articleId=1709224&page=1&viewMode=list&size=20
6   https://lineagem.plaync.com/board/server/view?articleId=1709223&page=1&viewMode=list&size=20
7   https://lineagem.plaync.com/board/server/view?articleId=1709222&page=1&viewMode=list&size=20
8   https://lineagem.plaync.com/board/server/view?articleId=1709221&page=1&viewMode=list&size=20
9   https://lineagem.plaync.com/board/free/view?articleId=1709220&page=1&viewMode=list&size=20
10   https://lineagem.plaync.com/board/image/view?articleId=1709219&page=1&viewMode=list&size=20
11   https://lineagem.plaync.c

86   https://lineagem.plaync.com/board/free/view?articleId=1709135&page=5&viewMode=list&size=20
87   https://lineagem.plaync.com/board/free/view?articleId=1709134&page=5&viewMode=list&size=20
88   https://lineagem.plaync.com/board/server/view?articleId=1709133&page=5&viewMode=list&size=20
89   https://lineagem.plaync.com/board/free/view?articleId=1709132&page=5&viewMode=list&size=20
90   https://lineagem.plaync.com/board/server/view?articleId=1709131&page=5&viewMode=list&size=20
91   https://lineagem.plaync.com/board/server/view?articleId=1709130&page=5&viewMode=list&size=20
92   https://lineagem.plaync.com/board/server/view?articleId=1709128&page=5&viewMode=list&size=20
93   https://lineagem.plaync.com/board/server/view?articleId=1709127&page=5&viewMode=list&size=20
94   https://lineagem.plaync.com/board/server/view?articleId=1709125&page=5&viewMode=list&size=20
95   https://lineagem.plaync.com/board/free/view?articleId=1709123&page=5&viewMode=list&size=20
96   https://lineagem.plaync

170   https://lineagem.plaync.com/board/server/view?articleId=1709034&page=9&viewMode=list&size=20
171   https://lineagem.plaync.com/board/free/view?articleId=1709033&page=9&viewMode=list&size=20
172   https://lineagem.plaync.com/board/free/view?articleId=1709032&page=9&viewMode=list&size=20
173   https://lineagem.plaync.com/board/image/view?articleId=1709031&page=9&viewMode=list&size=20
174   https://lineagem.plaync.com/board/free/view?articleId=1709028&page=9&viewMode=list&size=20
175   https://lineagem.plaync.com/board/server/view?articleId=1709027&page=9&viewMode=list&size=20
176   https://lineagem.plaync.com/board/server/view?articleId=1709026&page=9&viewMode=list&size=20
177   https://lineagem.plaync.com/board/server/view?articleId=1709025&page=9&viewMode=list&size=20
178   https://lineagem.plaync.com/board/free/view?articleId=1709023&page=9&viewMode=list&size=20
179   https://lineagem.plaync.com/board/server/view?articleId=1709021&page=9&viewMode=list&size=20
180   https://linea

In [7]:
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(3)

In [13]:
url = "https://lineagem.plaync.com/board/all/list"
driver.get(url)
sleep(1)

In [8]:
p = "div > c-wiz > div > div > div > div > div > div > div > div > c-wiz > div > div > div > div > div > span"
p2 = "div > c-wiz > div > div > div > div > div > div > div > div > c-wiz > div > div > div > div"

publicUrl = "#ncCommunityList > ul > li > div.title > a"
currentPage = "#ncCommunityList > div.pagination-container > ul > li.current"
title = "#ncCommunityView > section > div.view-header > h2"
body = "#ncCommunityView > section > div.view-body > div"
date = "span.date"
hits = "span.hit > em"
comments = "span.comment > em"
recommend = "#ncCommunityView > section > div.view-utils > button.co-btn.co-btn-like > em"

In [13]:
getUrl(12)

https://lineagem.plaync.com/board/free/view?articleId=1708778&page=1&viewMode=list&size=20
https://lineagem.plaync.com/board/free/view?articleId=1708778&page=1&viewMode=list&size=20
https://lineagem.plaync.com/board/free/view?articleId=1708755&page=2&viewMode=list&size=20
https://lineagem.plaync.com/board/free/view?articleId=1708755&page=2&viewMode=list&size=20
https://lineagem.plaync.com/board/server/view?articleId=1708754&page=2&viewMode=list&size=20
https://lineagem.plaync.com/board/server/view?articleId=1708754&page=2&viewMode=list&size=20
https://lineagem.plaync.com/board/free/view?articleId=1708734&page=3&viewMode=list&size=20
https://lineagem.plaync.com/board/free/view?articleId=1708734&page=3&viewMode=list&size=20
https://lineagem.plaync.com/board/server/view?articleId=1708733&page=3&viewMode=list&size=20
https://lineagem.plaync.com/board/server/view?articleId=1708733&page=3&viewMode=list&size=20
https://lineagem.plaync.com/board/image/view?articleId=1708732&page=3&viewMode=lis

['https://lineagem.plaync.com/board/free/view?articleId=1708800&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708799&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708798&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708797&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708796&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708795&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708794&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708792&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708791&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?articleId=1708790&page=1&viewMode=list&size=20',
 'https://lineagem.plaync.com/board/free/view?arti

In [79]:
driver.get(getUrl()[0])

In [ ]:
nextPage()

In [54]:
getTitle()

'접속 왜 안돼요?'

In [180]:
getBody()

'갑만에먹은파템이제 영인좀주세효몇번실패인지 ㅜㅜ'

In [161]:
getDate()

'2019-12-23'

In [162]:
getHits()

'292'

In [163]:
getRecommend()

'3'

In [22]:
from elasticsearch import Elasticsearch

class DataCollector:
    #publicUrl = "#ncCommunityList > ul > li > div.title > a"
    
    
    def __init__(self, name, host="localhost:9200"):
        self.es = Elasticsearch(hosts=host)
        self.name = name
        self.urls = []
        self.lasturl = ""
        self.publicUrl = "#ncCommunityList > ul > li > div.title > a"
        self.currentPage = "#ncCommunityList > div.pagination-container > ul > li.current"
        self.title = "#ncCommunityView > section > div.view-header > h2"
        self.body = "#ncCommunityView > section > div.view-body > div"
        self.date = "span.date"
        self.hits = "span.hit > em"
        self.recommend = "#ncCommunityView > section > div.view-utils > button.co-btn.co-btn-like > em"
        self.driver = webdriver.Chrome('chromedriver.exe')
        
    
    def parse(self, selector):
        return BeautifulSoup(self.driver.page_source, 'lxml', from_encoding='utf-8').select(selector)

    # 해당 Page 게시물 URL 전체 수집
    def getUrl(self):
        e = []
        for el in parse(self.publicUrl):
            e.append(el['href'])
        return e

    # 다음 Page로 이동
    def nextPage(self):
        num = int(parse(currentPage)[0].text)
        print(num+1)
        nextUrl = "https://lineagem.plaync.com/board/all/list?page="+str(num+1)
        print(nextUrl)
        self.driver.get(nextUrl)

    def getTitle(self):
        #print(getUrl()[0])
        #driver.get(getUrl()[0])
        return parse(title)[0].text

    def getBody(self):
        textBody = "";
        for s in parse(body):
            #print(s)
            textBody += s.text.replace("\xa0", "")
        return textBody

    def getDate(self):
        textDate = ""
        textDate = parse(date)[0].text
        return textDate.replace(" ","").replace(".","-")[:10]

    def getHits(self):
        return parse(hits)[0].text

    def getRecommend(self):
        return parse(recommend)[0].text
    
    def check_index(self, opt=True):
        if not opt and self.es.indices.exists(self.name):
            print("# 해당 인덱스 초기화를 위한 삭제")
            self.es.indices.delete(self.name)
        if not self.es.indices.exists(self.name):
            print("# 해당 인덱스 미존재")
            settings = {
                "index": {
                "analysis": {
                    "tokenizer": {
                    "nori_tokenizer_mixed": {
                        "type": "nori_tokenizer",
                        "decompound_mode": "mixed"
                    }
                    },
                    "analyzer": {
                    "korean": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer_mixed",
                        "filter": [
                        "nori_readingform",
                        "lowercase",
                        "nori_part_of_speech_basic"
                        ]
                    }
                    },
                    "filter": {
                    "nori_part_of_speech_basic": {
                        "type": "nori_part_of_speech",
                        "stoptags": ["E", "IC", "J",
                                    "MAG", "MAJ", "MM",
                                    "SP", "SSC", "SSO", "SC", "SE",
                                    "XPN", "XSA", "XSN", "XSV",
                                    "UNA", "NA", "VSV"]
                    }
                    }
                }
                }
            }
            body = {
            "settings": settings,
            "mappings" : {"properties" :
                            {"title" : {"type" : "text", "analyzer" : "korean"},
                                "body" : {"type" : "text", "analyzer" : "korean"},
                                "date" : {"type": "date", "format": "yyyy-MM-dd"}
                            }}
            }
            self.es.indices.create(index=self.name, body=body)#인덱스 생성
            print("# 해당 인덱스 생성 완료")
        
    def crawling():
        es = Elasticsearch(hosts="localhost:9200")
        test = []
        test2 = []
        for url in getUrl():
            driver.get(url)
            test.append({"create" : {"_index" : community_data, "_id" : url}})
            test.append({
                        "community" : "리니지M 공식홈페이지",
                        "title" : getTitle(),
                        "date" : getDate(),
                        "body" : getBody(),
                        "hits" : int(getHits().replace("," , "")),
                        "commments" : 0,
                        "recommand" : int(getRecommend().replace("," , ""))
                        })
        print(test)

    
    def scrape_bodies(self):
        url = "https://lineagem.plaync.com/board/all/list"
        self.driver.get(url)
        sleep(1)
        bulk_body = []
        for el in self.getUrl():    
            print(el)
            self.driver.get(el)
            sleep(0.1)
            bulk_body.append({"create" : {"_index" : self.name, "_id" : el}})
            soup = BeautifulSoup(requests.get(el).text, "lxml")
            bulk_body.append({
                        "community" : "리니지M 공식홈페이지",
                        "title" : getTitle(),
                        "date" : getDate(),
                        "body" : getBody(),
                        "hits" : int(getHits()),
                        "commments" : 0,
                        "recommand" : int(getRecommend())
                        })
            if  j < 11:
                self.es.bulk(index=self.name, body=bulk_body)
                self.es.indices.refresh(index=self.name)
                print("# 중간 저장")
                bulk_body.clear()
        #nextPage()        
#         if len(bulk_body) != 0:
#             self.es.bulk(index=self.name, body=bulk_body)
#             self.es.indices.refresh(index=self.name)
#         print("# 최종 저장") 



In [58]:
collector = DataCollector(name="community_data")
collector.check_index(opt=False)

#collector.scrape_urls("http://www.inven.co.kr/board/lineagem/5019", pages=1)
#collector.scrape_bodies()

NameError: name 'DataCollector' is not defined

In [5]:


import re, requests
class DataCollector:
    def __init__(self, name, host="192.168.1.3:9200"):
        self.es = Elasticsearch(hosts=host)
        self.name = name
        self.urls = []
        self.lasturl = ""
    def check_index(self, opt=True):
        if not opt and self.es.indices.exists(self.name):
            print("# 해당 인덱스 초기화를 위한 삭제")
            self.es.indices.delete(self.name)
        if not self.es.indices.exists(self.name):
            print("# 해당 인덱스 미존재")
            settings = {
                "index": {
                "analysis": {
                    "tokenizer": {
                    "nori_tokenizer_mixed": {
                        "type": "nori_tokenizer",
                        "decompound_mode": "mixed"
                    }
                    },
                    "analyzer": {
                    "korean": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer_mixed",
                        "filter": [
                        "nori_readingform",
                        "lowercase",
                        "nori_part_of_speech_basic"
                        ]
                    }
                    },
                    "filter": {
                    "nori_part_of_speech_basic": {
                        "type": "nori_part_of_speech",
                        "stoptags": ["E", "IC", "J",
                                    "MAG", "MAJ", "MM",
                                    "SP", "SSC", "SSO", "SC", "SE",
                                    "XPN", "XSA", "XSN", "XSV",
                                    "UNA", "NA", "VSV"]
                    }
                    }
                }
                }
            }
            body = {
            "settings": settings,
            "mappings" : {"properties" :
                            {"title" : {"type" : "text", "analyzer" : "korean"},
                                "body" : {"type" : "text", "analyzer" : "korean"},
                                "date" : {"type": "date", "format": "yyyy-MM-dd"}
                            }}
            }
            self.es.indices.create(index=self.name, body=body)#인덱스 생성
            print("# 해당 인덱스 생성 완료")

def scrape_bodies(self):
        if len(self.urls) == 0 :
            print("# 우선 URL 수집을 진행해주세요!")
            return
        bulk_body = []
        for i, el in enumerate(reversed(self.urls)):    
            print(i, el)
            sleep(0.1)
            bulk_body.append({"create" : {"_index" : self.name, "_id" : el}})
            soup = BeautifulSoup(requests.get(el).text, "lxml")
            bulk_body.append({
                        "community" : "리니지M 공식홈페이지",
                        "title" : soup.select_one(".articleTitle").text.strip(),
                        "date" : re.search(r'\d{4}-\d{2}-\d{2}', soup.select_one(".articleDate").text.strip()).group(),
                        "body" : re.sub(r'\xa0', "", soup.select_one("#powerbbsContent").text.strip()),
                        "hits" : int(re.search(r'[0-9,]+', soup.select_one(".articleHit").text).group().replace(",", "")),
                        "commments" : int(re.search(r'[0-9,]+', soup.select_one(".articleBottomMenu").text).group().replace(",", "")),
                        "recommand" : int(re.search(r'[0-9,]+', soup.select_one(".reqnum").text).group().replace(",", ""))
                        })
            if (i + 1) % 100 == 0:
                self.es.bulk(index=self.name, body=bulk_body)
                self.es.indices.refresh(index=self.name)
                print("# 중간 저장")
                bulk_body.clear()
        if len(bulk_body) != 0:
            self.es.bulk(index=self.name, body=bulk_body)
            self.es.indices.refresh(index=self.name)
        print("# 최종 저장") 

In [8]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


In [6]:
es = Elasticsearch(hosts="localhost:9200")
try:
    print(es.get(index="community_data", id="https://lineagem.plaync.com/board/server/view?articleId=1708330&page=9&viewMode=list&size=20")['_id'])
except:
    print("아이디가 없음")

https://lineagem.plaync.com/board/server/view?articleId=1708330&page=9&viewMode=list&size=20


In [ ]:
testUrl ="https://lineagem.plaync.com/board/free/view?articleId=1708518&page=9&viewMode=list&size=20" 
checkUrl(testUrl)
